## SVM

In [26]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import preprocessing
import matplotlib.pyplot as plt
import timeit

In [27]:
from preprocessing import cargarDatasets
from preprocessing import prepararSetDeEntrenamiento
from preprocessing import prepararSetDeValidacion
from preprocessing import ingenieriaDeFeaturesSVM

In [28]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split,KFold, StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score

In [29]:
from sklearn.svm import SVC

* Este metodo es un algortimo suvervisado en la cual busca un hiperplano que separae las clases de la mejor forma posible. 
* A su vez busca maximinar M (margen), el espacio entre los distintos puntos de las distintas clases. 
  Si se decea que los punos esten perfectamente separados tenemos un modelo hard margin, en donde el modelo no funciona con outliers y ruido. Si se le permite tener algo de rido y outliers entonces tenemos un modelo soft margin.

* En el caso en que no se encuentre el hiperplano, los puntos no son linealmente separables por lo que hay reccurir a una transformacion de datos en una nueva dimension. Como es costoso se utilizan kernes que hace que se piense que esta en otro dimension. Estos kernel puede sen lineal, polinomial, radial, etc.

## Carga y preprosesamiento de datos 

In [30]:
train_df,final_df = cargarDatasets()
train_df = prepararSetDeEntrenamiento(train_df)
final_df = prepararSetDeValidacion(final_df)

Una vez que tenemos cargados el set de entrenamiento vamos a ir aplicando distintos preprocesamientos

svm es sensible a la escala y distribucion de datos, es por eso que normalizamos las variables.

In [31]:
X,y,df,y_encoder = ingenieriaDeFeaturesSVM(train_df)

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=117, test_size=0.1, stratify=y)

## Busca de hiperparaemtros

los hiperparametros a tener en cuenta van a ser c y el kernel 
* c: es la resistencia al ruido, a mayor c se tiene un modelo hard margin y a menor c se tiene un modelo soft margin.
* kernel: es una funcion que nos permite tener dimensiones mayores sin tener que convertir los datos.     

In [33]:
def busquedaDeMejoresHiperparametros():
    mejor_puntaje: 0
    mejor_c: None
    mejor_kernel: None   
    for c in [1,10,20,30,40,50,60,70,80,90,100,150,200,300,400,500]:
        for kernel in ["poly", "rbf", "linear"]:
            kf = StratifiedKFold(n_splits=5)
            metricas = []
            for fold_idx, (train_index, test_index) in enumerate(kf.split(X_train, y_train)):
                svm = SVC(C = c, kernel = kernel, probability = True)
                svm.fit(X_train, y_train)
                predicciones = svm.predict_proba(X_train)
                puntaje = roc_auc_score(X_test,predicciones)
                metricas.append(puntaje)
            if np.mean(metricas) >= mejor_valor:
                mejor_puntaje = np.mean(metricas)
                mejor_c = c
                mejor_kernel = kernel
            
    return mejor_puntaje, mejor_c, mejor_kernel
    

In [ ]:
#mejor_valor, mejor_c, mejor_kernel = busquedaDeMejoresHiperparametros()

In [43]:
params = [ {'C': [1,10,20,30,40,50,60,70,80,90,100,150,200,300,400,500], 'kernel': ['linear']},
  {'C': [1,10,20,30,40,50,60,70,80,90,100,150,200,300,400,500], 'gamma': np.arange(1, 100, 10), 'kernel': ['rbf']},
   {'C': [1,10,20,30,40,50,60,70,80,90,100,150,200,300,400,500], 'gamma': np.arange(1, 100, 10), 
   'degree': np.arange(1, 100, 10),'coef0': np.arange(1, 100, 10),'kernel': ['poly']}]

In [44]:
#params = {'C': [1,10,20,30,40,50,60,70,80,90,100,150,200,300,400,500],'kernel' : ["poly", "rbf", "linear"]}

In [45]:
svm = SVC()
gscv = GridSearchCV(
    svm, params, scoring='roc_auc', n_jobs=-1, cv=5, return_train_score=True
).fit(X_train, y_train)
svm.fit(X_train, y_train

KeyboardInterrupt: 

In [ ]:
print(f"Best score: {gscv.best_score_}")
print(f"Best params {gscv.best_params_}")
   

Pruebo con uno solo 

In [34]:
%%timeit
svm = SVC(C = 1, kernel = 'linear', probability = True)
svm.fit(X_train, y_train)

2min 13s ± 5.62 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [35]:
y_pred = svm.predict(X_test)
accuracy_score(y_test, y_pred) 

0.8317470064476512

In [10]:
parametrosLineal = { 'C': [1,10,30], 'kernel' : ['linear'] }
parametrosRadial = { 'C': [1,10,20,30,40,50,60,70,80,90,100,150,200,300], 'gamma': np.arange(1, 80), 'kernel': ['rbf']}


In [11]:
svm = SVC()
gscv = GridSearchCV(
    svm, parametrosLineal, scoring='roc_auc', n_jobs=-1, cv=5, return_train_score=True
).fit(X_train, y_train)
svm.fit(X_train, y_train)

SVC()

In [12]:
print(f"Best score: {gscv.best_score_}")
print(f"Best params {gscv.best_params_}")

Best score: 0.863052420812388
Best params {'C': 30, 'kernel': 'linear'}


In [36]:
%%timeit
svm = SVC()
gscv = GridSearchCV(
    svm, parametrosRadial, scoring='roc_auc', n_jobs=-1, cv=5, return_train_score=True
).fit(X_train, y_train)
svm.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
print(f"Best score: {gscv.best_score_}")
print(f"Best params {gscv.best_params_}")